# Capstone Project - The Battle of the Neighborhoods (Week 2)

## Prepared By Anil Singh 
### Applied Data Science Capstone by IBM/Coursera


## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

PVR has a strong presence of movie theatres approx. 162 locations across 64 Indian cities. In Mumbai they have presence in 9 Location.

PVR Cinemas looking to open Burger Shops along with Movies Theaters. Based on Analysis we have to suggest which location will be cost-effective to start with.

Below are the 9 Locations that we are going to analysis:  Juhu  Mulund  Goregaon  Lower Parel  Kurla  Andheri  Panvel  Kandivali West  Dombivli

We looking to find the location which will be sutable based on customer base and less competition from other nearby Burger Shops.


In [1]:

import wikipedia as wp

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim 


from sklearn.cluster import KMeans
import folium

print('Required Liberary Imported.')

Required Liberary Imported.


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Data <a name="data"></a>

As a start point , we referred the data from Wikipedia to identify "PVR Cinemas“ locations. https://en.wikipedia.org/wiki/PVR_Cinemas

We got the list of all Movies theaters locality in India under "PVR Cinemas". Our objective to find locations in Mumbai - Maharashtra, so data cleaning activity is performed.

To find the geo locations of all Movies theaters locality we have used API of http://nominatim.openstreetmap.org

Later is Project , we used Foursquare to find the venues data based on nearby :

* Nearby other Multiplex & Movie Theaters

* Nearby other Burger Joint

* Nearby University & Colleges

* Nearby Office or working Locations


In [3]:
html = wp.page("PVR Cinemas").html().encode("UTF-8")
df = pd.read_html(html)[2]
df.head()

,State or territory,City,Theatre name,Theatre locality,Number of screens
0,National Capital Region (India),Delhi,PVR Priya,Vasant Vihar,01
1,National Capital Region (India),Delhi,PVR Rivoli,Connaught Place,01
2,National Capital Region (India),Delhi,PVR Shalimar Bagh,Shalimar Bagh,04
3,National Capital Region (India),Delhi,PVR Prashant Vihar,Rohini,03
4,National Capital Region (India),Delhi,PVR Select Citywalk ( IMAX & Gold),Saket,03


### Finding only Mumbai Location Data

In [4]:
df_Mumbai = df[df['City'] == "Mumbai"]

In [5]:

df = pd.DataFrame()
df['MumbaiVenue'] =df_Mumbai['Theatre locality']
df


,MumbaiVenue
106,Juhu
107,Mulund
108,Goregaon
109,Lower Parel
110,Kurla
111,Andheri
112,Panvel
113,Kandivali West
114,Dombivli


In [6]:
def get_coords_local(MumbaiVen, output_as='center'):
    """
   
        list with coordinates as str
    """
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             MumbaiVen+', Maharastra MUmbai',
                             '&format=json&polygon=0')
    response = requests.get(url).json()[0]
    
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output

In [7]:
df2 = df.copy()

latitudeCln = []
longitudeCln = []
for index, row in df2.iterrows():
    print(row[0])
    lat, long = get_coords_local(MumbaiVen=row[0], output_as='center')
    latitudeCln.append(lat)
    longitudeCln.append(long)

df2['Latitude'] = latitudeCln
df2['Longitude'] = longitudeCln

df2.shape

Juhu
Mulund
Goregaon
Lower Parel
Kurla
Andheri
Panvel
Kandivali West
Dombivli


(9, 3)

In [8]:
df2

,MumbaiVenue,Latitude,Longitude
106,Juhu,19.107021,72.827528
107,Mulund,19.172290,72.956469
108,Goregaon,19.164753,72.850018
109,Lower Parel,19.002988,72.830322
110,Kurla,19.062995,72.884714
111,Andheri,19.120371,72.848043
112,Panvel,18.989525,73.122193
113,Kandivali West,19.208380,72.842227
114,Dombivli,19.232921,73.032295


In [9]:
address = 'Mumbai, Maharastra'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude

In [10]:
df3 = df2.copy()
df3

,MumbaiVenue,Latitude,Longitude
106,Juhu,19.107021,72.827528
107,Mulund,19.172290,72.956469
108,Goregaon,19.164753,72.850018
109,Lower Parel,19.002988,72.830322
110,Kurla,19.062995,72.884714
111,Andheri,19.120371,72.848043
112,Panvel,18.989525,73.122193
113,Kandivali West,19.208380,72.842227
114,Dombivli,19.232921,73.032295


## Methodology <a name="methodology"></a>

#### In this project our efforts to analize all 9 locations and possbility of business based on the oppertunity for Burger Joint.


We have used location data for each 9 locations and further analyze all location based on following nearby opportunity & threads. We have used weight system in order to rate well the opportunity and subtract the threads and come out with fair mechanism to achieve the project goal and suggested the optimized location based of all below analysis


Opportunity Weights :
* Nearby Movie Theater (1.5)
* Nearby college_university(2.0)
* Nearby Working locations (2.5)

Threads Weights :
* Nearby other burger Joint (-2)


In [11]:
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, local in zip(df3['Latitude'], df3['Longitude'], df3['MumbaiVenue']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mumbai)  
    
map_mumbai

In [12]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            response = requests.get(url).json()
            results = response["response"]['venues']

            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['MumbaiVen', 
                  'MumbaiVen Latitude', 
                  'MumbaiVen Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [13]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'PYO0WJ21BZT1FNJX2D4M4Q3MQVT3HLVFMZGIPOPLLODORC2D'
CLIENT_SECRET = 'RTKMMIJ3C0W10GAPFGNUBMBMGOCI5EHZCN0NL0ZRXDAOMR5B'
VERSION = '20181020'

In [14]:
mumbai_pvr_burger = getNearbyVenues(names=df3['MumbaiVenue'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d16c941735')
mumbai_pvr_burger.head()

,MumbaiVen,MumbaiVen Latitude,MumbaiVen Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Juhu,19.107021,72.827528,Frisbees - The Burger Peddler,19.113349,72.828547,Burger Joint
1,Mulund,19.172290,72.956469,Kunj Vihar,19.172499,72.956256,Burger Joint
2,Mulund,19.172290,72.956469,McDonald's,19.168482,72.952225,Burger Joint
3,Mulund,19.172290,72.956469,kunjvihar mulund,19.170681,72.960431,Burger Joint
4,Mulund,19.172290,72.956469,Santosh Wada Pav,19.177024,72.964034,Burger Joint


In [15]:
mumbai_pvr_burger.shape

(26, 7)

In [16]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['MumbaiVen'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [17]:
map_pvr_burger = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(mumbai_pvr_burger, 'red', map_pvr_burger)
map_pvr_burger

In [18]:
mumbai_pvr_movie_theater = getNearbyVenues(names=df3['MumbaiVenue'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d17f941735')
mumbai_pvr_movie_theater.head()

,MumbaiVen,MumbaiVen Latitude,MumbaiVen Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Juhu,19.107021,72.827528,"juhu taxi stand , andheri",19.107115,72.826352,Movie Theater
1,Juhu,19.107021,72.827528,PVR Cinemas,19.114637,72.827452,Multiplex
2,Juhu,19.107021,72.827528,Pvr Audi 5,19.105800,72.825200,Multiplex
3,Juhu,19.107021,72.827528,pvr andheri,19.111589,72.830041,Multiplex
4,Juhu,19.107021,72.827528,"PVR Dynamix, Juhu",19.113196,72.826446,Multiplex


In [19]:
mumbai_pvr_movie_theater.shape

(59, 7)

In [20]:
map_mumbai_pvr_movie_theater = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(mumbai_pvr_movie_theater, 'green', map_mumbai_pvr_movie_theater)
map_mumbai_pvr_movie_theater

In [21]:
mumbai_pvr_college_university = getNearbyVenues(names=df3['MumbaiVenue'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4d4b7105d754a06372d81259')
mumbai_pvr_college_university.head()

,MumbaiVen,MumbaiVen Latitude,MumbaiVen Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Juhu,19.107021,72.827528,Jamnabai Narsee School,19.105950,72.832032,School
1,Juhu,19.107021,72.827528,Actors Prepare,19.109636,72.823942,College Administrative Building
2,Juhu,19.107021,72.827528,"St. Joseph's High School, Juhu",19.105534,72.826775,General College & University
3,Juhu,19.107021,72.827528,RIMS International School And Junior College,19.101637,72.826135,University
4,Juhu,19.107021,72.827528,Communication Lab,19.109027,72.837400,College Lab


In [22]:
mumbai_pvr_college_university.shape

(282, 7)

In [23]:
map_mumbai_pvr_college_university = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(mumbai_pvr_college_university, 'brown', map_mumbai_pvr_college_university)
map_mumbai_pvr_college_university

In [24]:
mumbai_pvr_office = getNearbyVenues(names=df3['MumbaiVenue'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d124941735')
mumbai_pvr_office.head()

,MumbaiVen,MumbaiVen Latitude,MumbaiVen Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Juhu,19.107021,72.827528,Citibank,19.104615,72.827485,Office
1,Juhu,19.107021,72.827528,Zomato,19.107493,72.829725,Office
2,Juhu,19.107021,72.827528,Punjab and Sinnd Bank Juhu Vile Parle,19.104105,72.830076,Office
3,Juhu,19.107021,72.827528,Kaya Skin Clinic- Juhu,19.109650,72.829807,Spa
4,Juhu,19.107021,72.827528,AB Corp Ltd,19.107489,72.826258,Advertising Agency


In [25]:
mumbai_pvr_office.shape

(324, 7)

In [26]:
map_mumbai_pvr_office = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(mumbai_pvr_office, 'yellow', map_mumbai_pvr_office)
map_mumbai_pvr_office

In [27]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('MumbaiVen').count()
    
    for n in startDf['MumbaiVen']:
        try:
            startDf.loc[startDf['MumbaiVen'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['MumbaiVen'] == n,columnTitle] = 0

In [28]:
df_data = df3.copy()
df_data.rename(columns={'MumbaiVenue':'MumbaiVen'}, inplace=True)
addColumn(df_data, 'Burger', mumbai_pvr_burger)
addColumn(df_data, 'Movie Theater', mumbai_pvr_movie_theater)
addColumn(df_data, 'College University', mumbai_pvr_college_university)
addColumn(df_data, 'Work Locations', mumbai_pvr_office)
df_data

,MumbaiVen,Latitude,Longitude,Burger,Movie Theater,College University,Work Locations
106,Juhu,19.107021,72.827528,1.0,17.0,45.0,43.0
107,Mulund,19.172290,72.956469,4.0,2.0,45.0,47.0
108,Goregaon,19.164753,72.850018,2.0,7.0,34.0,45.0
109,Lower Parel,19.002988,72.830322,9.0,14.0,35.0,50.0
110,Kurla,19.062995,72.884714,0.0,1.0,7.0,31.0
111,Andheri,19.120371,72.848043,4.0,9.0,48.0,47.0
112,Panvel,18.989525,73.122193,3.0,4.0,30.0,19.0
113,Kandivali West,19.208380,72.842227,3.0,5.0,38.0,41.0
114,Dombivli,19.232921,73.032295,0.0,0.0,0.0,1.0


In [29]:
burger_venue_wpoints = -2
movie_theater_venue_wpoints = 1.5
college_university_venue_wpoints = 2.0
office_venue_wpoints = 2.5

In [30]:
df_weighted = df_data[['MumbaiVen']].copy()

## Analysis <a name="analysis"></a>

In [31]:
df_weighted['Score'] = df_data['Burger'] * burger_venue_wpoints + df_data['Movie Theater'] * movie_theater_venue_wpoints + df_data['College University'] * college_university_venue_wpoints + df_data['Work Locations'] * office_venue_wpoints
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted

,MumbaiVen,Score
106,Juhu,221.0
111,Andheri,219.0
107,Mulund,202.5
109,Lower Parel,198.0
108,Goregaon,187.0
113,Kandivali West,180.0
112,Panvel,107.5
110,Kurla,93.0
114,Dombivli,2.5


In [32]:
address = 'Juhu, Mumbai'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude

In [33]:
map_pvr_result = folium.Map(location=[latitude, longitude], zoom_start=14)

pvr_burger_loc_win = df3[df3['MumbaiVenue'] == 'Juhu']

for lat, lng, local in zip(pvr_burger_loc_win['Latitude'], pvr_burger_loc_win['Longitude'], pvr_burger_loc_win['MumbaiVenue']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_pvr_result) 

addToMap(mumbai_pvr_burger[mumbai_pvr_burger['MumbaiVen'] == 'Juhu'], 'red', map_pvr_result)
addToMap(mumbai_pvr_movie_theater[mumbai_pvr_movie_theater['MumbaiVen'] == 'Juhu'], 'green', map_pvr_result)
addToMap(mumbai_pvr_college_university[mumbai_pvr_college_university['MumbaiVen'] == 'Juhu'], 'brown', map_pvr_result)
addToMap(mumbai_pvr_office[mumbai_pvr_office['MumbaiVen'] == 'Juhu'], 'yellow', map_pvr_result)

map_pvr_result

## Results and Discussion <a name="results"></a>

### Juhu location clearly showing more number of business opportunity compare to any other locations.

### Business opportunity in Juhu
* 17 Movies Theatres
* 45  Colleges & University
* 43 Work locations

### Business threads / competitions in Juhu
* Only 1 Burger Joint is operating 

## Conclusion <a name="conclusion"></a>

### Based on our weightage System raking , Juhu location score highest between all 9 locations. 
